In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np


In [3]:
file_path = 'TRAINING_DATA.txt'

# Initialize lists to store labels and sentences
labels = []
sentences = []

# Read the text file line by line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Split each line into label and sentence using tab as the delimiter
        label, sentence = line.strip().split('\t')
        labels.append(int(label))
        sentences.append(sentence)

# Create a DataFrame
data = pd.DataFrame({'label': labels, 'text': sentences})

# Display the first few rows of the DataFrame
print(data.head())


   label                                               text
0      1  Cuando conocí a Janice en 2013 , una familia n...
1      0  Hwang habló en Sur de este año por Southwest M...
2      1  Usted podría pensar Katy Perry y Robert Pattin...
3      1  Cualquiera que haya volado los cielos del crea...
4      1  Bueno , este cantante tendrá un LARGO tiempo p...


In [4]:
data.head()

,label,text
0,1,"Cuando conocí a Janice en 2013 , una familia n..."
1,0,Hwang habló en Sur de este año por Southwest M...
2,1,Usted podría pensar Katy Perry y Robert Pattin...
3,1,Cualquiera que haya volado los cielos del crea...
4,1,"Bueno , este cantante tendrá un LARGO tiempo p..."


In [5]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw to /root/nltk_data...


True

In [6]:
pip install transformers

GRADIENT BOOST WITH TF-IDF

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
import string
from nltk.tokenize import word_tokenize

from transformers import AutoTokenizer, AutoModel
import torch
# stopwords
#nltk.download('stopwords')

# Define a function to preprocess text
def preprocess_text(text):
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

# Apply preprocessing to the dataset
data['text'] = data['text'].apply(preprocess_text)

# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# Function to get sentence embeddings in batches
def get_embeddings_in_batches(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # Use the embeddings for [CLS] token
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Get embeddings for train and test sets
X_train_embeddings = get_embeddings_in_batches(X_train.tolist())
X_test_embeddings = get_embeddings_in_batches(X_test.tolist())

# Define and train the classifier
classifier = GradientBoostingClassifier(n_estimators=200, learning_rate=0.01)
classifier.fit(X_train_embeddings, y_train)

# Evaluate the classifier
accuracy_gb_test = classifier.score(X_test_embeddings, y_test)
print("Accuracy with Embeddings on Test Set (Gradient Boosting):", accuracy_gb_test)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy with Embeddings on Test Set (Gradient Boosting): 0.6593959731543624


DEALING WITH THE REAL DATA

In [10]:
file_path = 'REAL_DATA.txt'

# Initialize lists to store labels and sentences
labels = []
sentences = []

# Read the text file line by line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Split each line into label and sentence using tab as the delimiter
        label, sentence = line.strip().split('\t')
        labels.append(int(label))
        sentences.append(sentence)

# Create a DataFrame
real_data = pd.DataFrame({'label': labels, 'text': sentences})

# Display the first few rows of the DataFrame
print(real_data.head())




   label                                               text
0      2  Yo no creo que a nadie le haya encantado un pe...
1      2  No va a resolver sus problemas de crédito o me...
2      2                                Te encantará este !
3      2  Yo estaba a volar a un aeropuerto varias horas...
4      2  ( Maid En Manhattan , The Wedding Planner , Je...


In [12]:
# Apply preprocessing to the unseen dataset
real_data['text'] = real_data['text'].apply(preprocess_text)

# Get embeddings for the unseen data
X_real_embeddings = get_embeddings_in_batches(real_data['text'].tolist())

# Predict labels for the unseen data
predictions = classifier.predict(X_real_embeddings)

# Add predictions to the DataFrame
real_data['predicted_label'] = predictions

# Display the first few rows of the DataFrame with predictions
print(real_data.head())


   label                                               text  \
0      2  Yo no creo que a nadie le haya encantado un pe...   
1      2  No va a resolver sus problemas de crédito o me...   
2      2                                 Te encantará este    
3      2  Yo estaba a volar a un aeropuerto varias horas...   
4      2   Maid En Manhattan  The Wedding Planner  Jerse...   

                                      text_processed  predicted_label  
0  Yo no creo que a nadie le haya encantado un pe...                1  
1  No va a resolver sus problemas de crédito o me...                0  
2                                 Te encantará este                 1  
3  Yo estaba a volar a un aeropuerto varias horas...                0  
4   Maid En Manhattan  The Wedding Planner  Jerse...                0  


In [13]:
real_data.head()

,label,text,text_processed,predicted_label
0,2,Yo no creo que a nadie le haya encantado un pe...,Yo no creo que a nadie le haya encantado un pe...,1
1,2,No va a resolver sus problemas de crédito o me...,No va a resolver sus problemas de crédito o me...,0
2,2,Te encantará este,Te encantará este,1
3,2,Yo estaba a volar a un aeropuerto varias horas...,Yo estaba a volar a un aeropuerto varias horas...,0
4,2,Maid En Manhattan The Wedding Planner Jerse...,Maid En Manhattan The Wedding Planner Jerse...,0


In [14]:
real_data.drop(columns=['text_processed'],inplace=True)
real_data.head()

,label,text,predicted_label
0,2,Yo no creo que a nadie le haya encantado un pe...,1
1,2,No va a resolver sus problemas de crédito o me...,0
2,2,Te encantará este,1
3,2,Yo estaba a volar a un aeropuerto varias horas...,0
4,2,Maid En Manhattan The Wedding Planner Jerse...,0


In [15]:
# Save DataFrame to a text file with tab-separated values
real_data.to_csv('REAL_DATA.txt', sep='\t', index=False, header=False)

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
